In [1]:
import numpy as np 
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
model = open(r'C:\Users\KIIT\Documents\LGM-Soc contributions\Uber Fare Prediction\Model/rf_model.pkl','rb')
forest = pickle.load(model)

In [3]:
test_df = pd.read_csv(r'C:\Users\KIIT\Documents\LGM-Soc contributions\Uber Fare Prediction\Dataset/test.csv')

In [4]:
test_df

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1
...,...,...,...,...,...,...,...
9909,2015-05-10 12:37:51.0000002,2015-05-10 12:37:51 UTC,-73.968124,40.796997,-73.955643,40.780388,6
9910,2015-01-12 17:05:51.0000001,2015-01-12 17:05:51 UTC,-73.945511,40.803600,-73.960213,40.776371,6
9911,2015-04-19 20:44:15.0000001,2015-04-19 20:44:15 UTC,-73.991600,40.726608,-73.789742,40.647011,6
9912,2015-01-31 01:05:19.0000005,2015-01-31 01:05:19 UTC,-73.985573,40.735432,-73.939178,40.801731,6


In [5]:
test_df.dropna(inplace=True)
test_df.isnull().sum()

key                  0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [6]:
def change_into_datetime(col):
    test_df[col] = pd.to_datetime(test_df[col])

In [7]:
for i in ['key','pickup_datetime']:
    change_into_datetime(i)

In [8]:
test_df['pickup_year'] = test_df['pickup_datetime'].dt.year
test_df['pickup_month'] = test_df['pickup_datetime'].dt.month
test_df['pickup_date'] = test_df['pickup_datetime'].dt.day
test_df['pickup_day_of_month'] = test_df['pickup_datetime'].dt.dayofweek

In [9]:
def extract_hour(test_df,col):
    test_df[col + '_hour'] = test_df[col].dt.hour

def extract_minute(train_df,col):
    test_df[col + '_minute'] = test_df[col].dt.minute

def extract_second(train_df,col):
    test_df[col + '_second'] = test_df[col].dt.second

In [10]:
extract_hour(test_df,'pickup_datetime')
extract_minute(test_df,'pickup_datetime')
extract_second(test_df,'pickup_datetime')

In [11]:
test_df.drop(columns=['key','pickup_datetime'], axis=1, inplace=True)

In [12]:
test_df.drop(test_df[test_df['pickup_latitude'] < -90].index, axis = 0, inplace = True)
test_df.drop(test_df[test_df['pickup_latitude'] > 90].index, axis = 0, inplace = True)

test_df.drop(test_df[test_df['dropoff_latitude'] < -90].index, axis = 0, inplace = True)
test_df.drop(test_df[test_df['dropoff_latitude'] > 90].index, axis = 0, inplace = True)

In [13]:
test_df.drop(test_df[test_df['pickup_longitude'] < -180].index, axis = 0, inplace = True)
test_df.drop(test_df[test_df['pickup_longitude'] > 180].index, axis = 0, inplace = True)

test_df.drop(test_df[test_df['dropoff_longitude'] < -180].index, axis = 0, inplace = True)
test_df.drop(test_df[test_df['dropoff_longitude'] > 180].index, axis = 0, inplace = True)

In [14]:
def haversine_distance(pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude):
    
    data = [test_df]
    
    for i in data:
        R = 6371 # radius of the erth in kilometers
        phi1 = np.radians(i[pickup_latitude])
        phi2 = np.radians(i[dropoff_latitude])
        
        delta_phi = np.radians(i[dropoff_latitude] - i[pickup_latitude])
        delta_lambda = np.radians(i[dropoff_longitude] - i[pickup_longitude])
        
        a = np.sin(delta_phi / 2.0) **2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
        
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        
        d = R * c
        
        i['harvesine_distance'] = d
        
    return d 

In [15]:
haversine_distance('pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude')

0        2.323260
1        2.425353
2        0.618628
3        1.961033
4        5.387301
          ...    
9909     2.124874
9910     3.270969
9911    19.183941
9912     8.343486
9913     1.180825
Length: 9914, dtype: float64

In [16]:
test_df.drop(test_df[test_df['passenger_count'] > 6].index, axis = 0,inplace = True)

In [17]:
test_df

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_date,pickup_day_of_month,pickup_datetime_hour,pickup_datetime_minute,pickup_datetime_second,harvesine_distance
0,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,1,13,8,24,2.323260
1,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,1,13,8,24,2.425353
2,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,5,11,53,44,0.618628
3,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,5,21,12,12,1.961033
4,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,5,21,12,12,5.387301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9909,-73.968124,40.796997,-73.955643,40.780388,6,2015,5,10,6,12,37,51,2.124874
9910,-73.945511,40.803600,-73.960213,40.776371,6,2015,1,12,0,17,5,51,3.270969
9911,-73.991600,40.726608,-73.789742,40.647011,6,2015,4,19,6,20,44,15,19.183941
9912,-73.985573,40.735432,-73.939178,40.801731,6,2015,1,31,5,1,5,19,8.343486


In [18]:
test_df.shape

(9914, 13)

In [19]:
test_preds = forest.predict(test_df)

In [20]:
test_preds

array([10.361654  , 10.012609  ,  4.3954535 , ...,  8.52553571,
       21.1289689 ,  7.32224525])

In [21]:
test_df['Predicted Fare'] = test_preds

In [22]:
test_df

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_date,pickup_day_of_month,pickup_datetime_hour,pickup_datetime_minute,pickup_datetime_second,harvesine_distance,Predicted Fare
0,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,1,13,8,24,2.323260,10.361654
1,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,1,13,8,24,2.425353,10.012609
2,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,5,11,53,44,0.618628,4.395453
3,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,5,21,12,12,1.961033,9.244239
4,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,5,21,12,12,5.387301,16.392996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9909,-73.968124,40.796997,-73.955643,40.780388,6,2015,5,10,6,12,37,51,2.124874,8.331904
9910,-73.945511,40.803600,-73.960213,40.776371,6,2015,1,12,0,17,5,51,3.270969,13.202330
9911,-73.991600,40.726608,-73.789742,40.647011,6,2015,4,19,6,20,44,15,19.183941,8.525536
9912,-73.985573,40.735432,-73.939178,40.801731,6,2015,1,31,5,1,5,19,8.343486,21.128969


In [23]:
test_df.to_excel(r'C:\Users\KIIT\Documents\LGM-Soc contributions\Uber Fare Prediction\Dataset\test_predictions.xlsx')